# Quantum Blockchain Demo (Colab)

This notebook demonstrates running the repo code in Google Colab, includes small benchmarks comparing old vs new Python hashing behavior, and provides simple simulator-only examples for Qiskit and Cirq.

Notes:
- This notebook clones the repository and installs the required packages.
- Qiskit and Cirq are installed for simulator examples only (no hardware calls).


In [ ]:
# Install dependencies (may take a few minutes)
!pip install -r requirements.txt --quiet

In [ ]:
# Clone the repository so we can run local modules
!git clone https://github.com/Deluciferrral/quantum-blockchain-Genesis.git repo_clone --quiet || true
%cd repo_clone

In [ ]:
# Simple hash functions to compare old vs new behavior
import hashlib, time

def old_hash(block_string):
    # Original behaviour simulated in repo: 1 SHA256 + 3 additional iterations with index mixing
    h = hashlib.sha256(block_string.encode()).hexdigest()
    for i in range(3):
        h = hashlib.sha256((h + str(i)).encode()).hexdigest()
    return h

def new_hash(block_string):
    # New behaviour in repo: 2 rounds of SHA256
    h = hashlib.sha256(block_string.encode()).hexdigest()
    h = hashlib.sha256(h.encode()).hexdigest()
    return h

block = '0previous1234567890timestamp{}data{}'.format(time.time(), 'example')

# Quick benchmark
for fn,name in [(old_hash,'old_hash (4 rounds)'), (new_hash,'new_hash (2 rounds)')]:
    t0 = time.time()
    for _ in range(500):
        fn(block)
    t1 = time.time()
    print(f'{name}: {t1-t0:.4f}s for 500 iterations')

# Print sample hashes
print('old_hash sample:', old_hash(block))
print('new_hash sample:', new_hash(block))


## Qiskit simulator example (no hardware)
This example builds a trivial 1-qubit circuit and runs it on the Aer simulator.


In [ ]:
# Qiskit example
try:
    from qiskit import QuantumCircuit, Aer, execute
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure(0,0)
    backend = Aer.get_backend('aer_simulator')
    job = execute(qc, backend=backend, shots=1024)
    result = job.result()
    counts = result.get_counts()
    print('Qiskit counts:', counts)
except Exception as e:
    print('Qiskit example failed:', e)


## Cirq simulator example (no hardware)
This example builds a trivial 1-qubit circuit and runs it on the Cirq simulator.


In [ ]:
# Cirq example
try:
    import cirq
    q = cirq.LineQubit(0)
    circuit = cirq.Circuit(cirq.H(q), cirq.measure(q, key='m'))
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1000)
    print('Cirq result keys:', result.histogram(key='m'))
except Exception as e:
    print('Cirq example failed:', e)


## Notes on Real Quantum Hardware
- This notebook uses local simulators only.
- To access real quantum hardware, follow each provider's instructions and add API credentials as environment variables.
- Do NOT store API keys directly in the notebook.
